***
### 旅行商模型——比赛项目排序问题
> 2005年电工杯B题

引入虚拟项目15作为开始和结束项目，则问题可以采用TSP求解，在找到的最短回路中，去掉虚拟项目15关联的两条权重为0的边即可。

In [1]:
import cvxpy as cp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif']=['Times New Roman + SimSun + WFM Sans SC']
plt.rcParams['mathtext.fontset']='stix'
# Times New Roman + SimSun + WFM Sans SC
# simsum宋体, times new roman -*, simhei黑体, kaiti楷体, 
# dengxian等线, fangsong仿宋, Microsoft Yahei微软雅黑
plt.rcParams['axes.unicode_minus']=False   
plt.rcParams['figure.dpi'] = 200
# plt.rcParams['figure.figsize'] = [4, 3]
# plt.rcParams['font.size'] = 12
plt.rcParams['xtick.direction']='in'
plt.rcParams['ytick.direction']='in'

In [2]:
df = pd.read_excel('../04第4章  线性规划和整数规划模型/data4_13_1.xlsx', header=None)
a = df.values
a[np.isnan(a)] = 0
a

array([[0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0.],
       [0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1.],
       [0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1.],
       [0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0.],
       [1., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 1.],
       [0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0

In [3]:
m, n = a.shape
infty = 1e7
w = np.zeros((n+1, n+1))
for i in range(n):
    for j in range(n):
        if i == j:
            w[i, j] = infty
        else:
            w[i, j] = np.sum(a[:, i]*a[:, j])
w[n, n] = infty
w

array([[1.e+07, 2.e+00, 2.e+00, 1.e+00, 0.e+00, 0.e+00, 1.e+00, 0.e+00,
        1.e+00, 2.e+00, 1.e+00, 1.e+00, 1.e+00, 1.e+00, 0.e+00],
       [2.e+00, 1.e+07, 1.e+00, 4.e+00, 1.e+00, 0.e+00, 1.e+00, 1.e+00,
        1.e+00, 3.e+00, 1.e+00, 0.e+00, 2.e+00, 1.e+00, 0.e+00],
       [2.e+00, 1.e+00, 1.e+07, 1.e+00, 0.e+00, 0.e+00, 0.e+00, 3.e+00,
        1.e+00, 1.e+00, 0.e+00, 2.e+00, 2.e+00, 1.e+00, 0.e+00],
       [1.e+00, 4.e+00, 1.e+00, 1.e+07, 2.e+00, 1.e+00, 2.e+00, 1.e+00,
        0.e+00, 2.e+00, 1.e+00, 1.e+00, 1.e+00, 1.e+00, 0.e+00],
       [0.e+00, 1.e+00, 0.e+00, 2.e+00, 1.e+07, 1.e+00, 0.e+00, 1.e+00,
        1.e+00, 1.e+00, 0.e+00, 1.e+00, 1.e+00, 2.e+00, 0.e+00],
       [0.e+00, 0.e+00, 0.e+00, 1.e+00, 1.e+00, 1.e+07, 1.e+00, 2.e+00,
        1.e+00, 1.e+00, 1.e+00, 1.e+00, 1.e+00, 2.e+00, 0.e+00],
       [1.e+00, 1.e+00, 0.e+00, 2.e+00, 0.e+00, 1.e+00, 1.e+07, 1.e+00,
        1.e+00, 1.e+00, 0.e+00, 2.e+00, 2.e+00, 1.e+00, 0.e+00],
       [0.e+00, 1.e+00, 3.e+00, 1.e+00, 1

In [4]:
x = cp.Variable((n+1, n+1), integer=True)
u = cp.Variable(n+1, integer=True)
obj = cp.Minimize(cp.sum(cp.multiply(w, x)))
cons = [
    cp.sum(x, axis=0) == 1,
    cp.sum(x, axis=1) == 1,
    u[0] == 0,
    1 <= u[1:], u[1:] <= n,
    0 <= x, x <= 1
]
for i in range(n+1):
    for j in range(1, n+1):
        cons.append(u[i] - u[j] + (n+1)*x[i, j] <= n)
prob = cp.Problem(obj, cons)
prob.solve(solver='GLPK_MI')
print(f'最优解为：{x.value}'), print(f'最优值为：{prob.value}')

最优解为：[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
最优值为：2.0


(None, None)

In [5]:
i, j = np.nonzero(x.value)
print(i+1)
print(j+1)

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
[ 5 14  6  9 11  2  3  1  8 13  7 10 15 12  4]


> 所取的infty不同，最终结果居然也不同
> 
> ***！！注意 ！！***
> 
> 因为cvxpy求解规划问题本质是在做数值计算，所以如果infty取得过大可能导致其他变量的值被舍去，因此定义infty时只需比其他值大2~3个数量级即可